<a href="https://colab.research.google.com/github/Yasaman-habibi/Pre_Processing_Report/blob/main/Heat_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Library

import os
import re
import nltk
import glob
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive, files
from google.colab import drive
import ipywidgets as widgets
import seaborn as sns
import numpy as np
import plotly.express as px

In [ ]:
#Upload Files

drive.mount('/content/drive')
source_path = "/content/drive/MyDrive/Combined_Texts"
all_txt_files = glob.glob(os.path.join(source_path, "*.txt"))

selector = widgets.SelectMultiple(
    options=all_txt_files,
    description='Select files',
    rows=10
)
display(selector)

In [ ]:
uploaded_Texts = list(selector.value)
print(" انتخاب شد:\n" + "\n".join(uploaded_Texts))

In [4]:
# Heatmap with Percent

import os
import re
import pandas as pd
import plotly.express as px

def process_heatmap_tagged(uploaded_Texts, sustain_terms, Heatmap_path):
    records = []

    sustain_terms = sorted(set([kw.strip().lower() for kw in sustain_terms]),
                           key=lambda x: len(x.split()), reverse=True)

    for file_path in uploaded_Texts:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        pattern = r"===== Start of File: (.+?) =====\n(.*?)\n===== End of File: \1 ====="
        reports = re.findall(pattern, content, flags=re.DOTALL)

        for report_name, report_text in reports:
            text = report_text.lower()
            counts = {}

            for term in sustain_terms:
                term_pattern = r"\b" + re.escape(term) + r"\b"
                matches = list(re.finditer(term_pattern, text))
                if matches:
                    counts[term] = len(matches)
                    text = re.sub(term_pattern, " " * len(term), text)

            for kw, cnt in counts.items():
                records.append({"report_name": report_name, "keyword": kw, "count": cnt})

    # Create DataFrame
    df = pd.DataFrame(records)
    if df.empty:
        print(" هیچ کلیدواژه‌ای پیدا نشد.")
        return df

    os.makedirs(Heatmap_path, exist_ok=True)

    df["Year"] = df["report_name"].str.extract(r"(\d{4})")
    df["Year"] = df["Year"].fillna("Unknown")


    # Calculate Heatmap by Percent
    year_totals = df.groupby("Year")["count"].sum().to_dict()
    df["percent"] = df.apply(lambda row: (row["count"] / year_totals[row["Year"]]) * 100
                             if row["Year"] in year_totals else 0, axis=1)

    # Create Heatmap
    heatmap_all = df.pivot_table(
        index="keyword", columns="Year", values="percent",
        aggfunc="sum", fill_value=0
    )

    fig_all = px.imshow(
        heatmap_all,
        text_auto=".1f",
        aspect="auto",
        color_continuous_scale="tealrose",
        labels=dict(x="Year", y="Keyword", color="Percent")
    )
    fig_all.update_layout(title="Heatmap of Keywords by Year (in %)")

    fig_all.write_html(os.path.join(Heatmap_path, "heatmap_all_reports.html"))
    fig_all.show()


    # Create Heatmap year by year
    for year, sub_df in df.groupby("Year"):
        year_heatmap = sub_df.pivot_table(
            index="keyword", columns="Year", values="percent",
            aggfunc="sum", fill_value=0
        )

        if not year_heatmap.empty:
            fig_year = px.imshow(
                year_heatmap,
                text_auto=".1f",
                aspect="auto",
                color_continuous_scale="Mint",
                labels=dict(x="Year", y="Keyword", color="Percent")
            )
            fig_year.update_layout(title=f"Heatmap for Year {year} (in %)")

            fig_year.write_html(os.path.join(Heatmap_path, f"heatmap_{year}.html"))
            fig_year.show()

    print(" Heatmap کلی و سال به سال با درصد ساخته شد .")
    return df

In [5]:
sustain_Dic = pd.read_excel("/content/drive/MyDrive/sustainability_table/PivotReport/keywords.xlsx")
keywords = sustain_Dic["keyword"].dropna().str.strip().tolist()

Heatmap_path = "/content/drive/MyDrive/sustainability_table/Heatmap"
os.makedirs(Heatmap_path, exist_ok=True)

In [ ]:
df_result = process_heatmap_tagged(uploaded_Texts, keywords, Heatmap_path)
